## Hangman Logic

In [9]:
def hangman(secret_word: str, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """Play a hangman game with the secret word with the provided guesser"""

    # Initialize the game state
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is",
              ' '.join(mask), 'length', len(secret_word))

    # Loop through the game
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        # Print the guess
        if verbose:
            print('Guess is', guess)

        # Check if the guess is already guessed
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            # Add the guess to the guessed set if it is valid and not already guessed
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                # Print the mistake if the guess is invalid
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1

        # Check if the game is over and break
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes

    # Print the secret word if the game is lost
    if verbose:
        print('Out of guesses. The word was', secret_word)
    # Return the number of mistakes made
    return mistakes

## Preprocessing

In [10]:
import nltk
import numpy as np
from nltk.corpus import brown


def preprocess(corpus):
    """Preprocess and shuffle the corpus"""

    corpus = set([word.lower() for word in corpus])
    corpus = list([word for word in corpus if word.isalpha()])
    np.random.shuffle(corpus)
    return corpus


# Download the Brown corpus
nltk.download('brown')
np.random.seed(1)

# Preprocess and split the corpus
brown_corpus = preprocess(brown.words())
training_set = brown_corpus[1000:]
test_set = brown_corpus[:1000]
print("Number of word types in train =", len(training_set))
print("Number of word types in test =", len(test_set))

[nltk_data] Downloading package brown to /home/vscode/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Number of word types in train = 39234
Number of word types in test = 1000


## Human Guesser

In [11]:
def human(mask, guessed, **kwargs):
    """Play a hangman game interactively with the user"""

    print('\nEnter your guess:')
    return input().lower().strip()


# Set `interactive` to `True` to play the game manually
interactive = False
# interactive = True

if interactive:
    hangman('whatever', human, 8, True)